In [135]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import math
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR


In [136]:
whole_data = pd.read_csv('whole_data.csv')

In [137]:
def normalize_isochron_group(group, x:str, y: str, new_column):
    min_max_scaler = preprocessing.MinMaxScaler()
    group[new_column] = group.apply(lambda row: 0 if row[y] == 0 else row[x] / row[y], axis=1)
    group[new_column + '_normalized'] = min_max_scaler.fit_transform(group[new_column].values.reshape(-1, 1))
    group.drop(new_column, axis=1, inplace=True)
    return group

def normalize_isochron_minus_group(group, x:str, y: str, new_column):
    min_max_scaler = preprocessing.MinMaxScaler()
    group[new_column] = group.apply(lambda row: row[x] - row[y], axis=1)
    group[new_column + '_normalized'] = min_max_scaler.fit_transform(group[new_column].values.reshape(-1, 1))
    group.drop(new_column, axis=1, inplace=True)
    return group

def derivative(group, x:str, y: str, new_column):
    min_max_scaler = preprocessing.MinMaxScaler()
    group[new_column] = group.apply(lambda row: 0 if row[y] == 0 else (row[x] - row[y]) / row[y], axis=1)
    #group[new_column + '_normalized'] = min_max_scaler.fit_transform(group[new_column].values.reshape(-1, 1)) # uncomment this and one row below if you want normalized derivative
    #group.drop(new_column, axis=1, inplace=True)
    return group

def normalize_group(group, x:str):
    min_max_scaler = preprocessing.MinMaxScaler()
    group[x + '_normalized'] = min_max_scaler.fit_transform(group[x].values.reshape(-1, 1))
    group.drop(x, axis=1, inplace=True)
    return group

In [138]:
set1 = set(whole_data[whole_data['type'] =='test'].region)
set2 = set(whole_data[whole_data['type'] =='train'].region)

difference = set2.difference(set1)
print("Difference between set1 and set2:", difference)

Difference between set1 and set2: {'Кызылординская область', 'Туркестанская область', 'Ханты-Мансийский автономный округ'}


In [139]:

# Новые признаки из базовых. Отношения, скорость
for x, y in zip(['people_iso_15', 'people_iso_10', 'people_iso_15', 'people_iso_3', 'people_iso_10', 'people_iso_5', 'people_iso_3', 'people_iso_5', 'people_iso_3', 'people_iso_3'],
                ['people_iso_30', 'people_iso_30', 'people_iso_30', 'people_iso_30', 'people_iso_15', 'people_iso_15', 'people_iso_15', 'people_iso_10', 'people_iso_10', 'people_iso_5']):
    whole_data = whole_data.groupby('region').apply(normalize_isochron_group, x, y, x + '_on_' + y)
for x, y in zip(['people_iso_30', 'people_iso_15', 'people_iso_10', 'people_iso_10', 'people_iso_5'], ['people_iso_15', 'people_iso_10', 'people_iso_5', 'people_iso_3', 'people_iso_3']):
    whole_data = whole_data.groupby('region').apply(normalize_isochron_minus_group, x, y, x + '_min_' + y)
    
for x, y in zip(['people_iso_30', 'people_iso_15', 'people_iso_10', 'people_iso_10', 'people_iso_5'], ['people_iso_15', 'people_iso_10', 'people_iso_5', 'people_iso_3', 'people_iso_3']):
    whole_data = whole_data.groupby('region').apply(derivative, x, y, x + '_deriv_' + y)
    
for x in ['people_iso_30', 'people_iso_15', 'people_iso_10', 'people_iso_5',
          'people_iso_3', 'count_pvz_iso_3', 'count_pvz_iso_5', 'count_pvz_iso_10', 'count_pvz_iso_15', 'count_pvz_iso_30',
          'pop_per_pvz_iso3', 'pop_per_pvz_iso5', 'pop_per_pvz_iso10', 'pop_per_pvz_iso15', 'pop_per_pvz_iso30', 
          'Nearest_Neighbor', 'Distance', 'commercial_density_300', 'commercial_density_500', 
          'count_railway_station_iso_3', 'count_railway_station_iso_5', 'count_railway_station_iso_10', 'count_railway_station_iso_15', 'count_railway_station_iso_30']:
    whole_data = whole_data.groupby('region').apply(normalize_group, x)



In [140]:
# distribution std by group
std_per_group = whole_data.groupby('region')['sales_per_month'].std()
std_whole_data = whole_data['sales_per_month'].std()

# Create a new feature 'z' based on the standard deviation ratio for each group
whole_data['std_ratio'] = whole_data.apply(lambda row: 1 if ((std_per_group[row['region']] == 0) or (np.isnan(std_per_group[row['region']])) ) else std_per_group[row['region']] / std_whole_data, axis=1)


In [141]:
whole_data.drop('geometry', axis = 1, inplace=True)
whole_data.drop('index_right', axis = 1, inplace=True)

In [142]:
test = whole_data[whole_data['type'] =='test']
data = whole_data[whole_data['type'] =='train']

In [143]:
#test.drop('Nearest_Neighbor', axis = 1, inplace=True)
test.drop('type', axis = 1, inplace=True)
test.drop('sales_per_month', axis = 1, inplace=True)
test

<ipython-input-143-7ddd8cc986bc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('type', axis = 1, inplace=True)
<ipython-input-143-7ddd8cc986bc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.drop('sales_per_month', axis = 1, inplace=True)


,id,region,locality,x,y,fclass,fclass_encoded,locality_encoded,region_encoded,Population,...,Nearest_Neighbor_normalized,Distance_normalized,commercial_density_300_normalized,commercial_density_500_normalized,count_railway_station_iso_3_normalized,count_railway_station_iso_5_normalized,count_railway_station_iso_10_normalized,count_railway_station_iso_15_normalized,count_railway_station_iso_30_normalized,std_ratio
8586,8319,Московская область,Домодедово,37.762291,55.450032,residential,12,301,57,7768878,...,0.296828,0.042388,0.175258,0.134503,0.0,0.0,0.0,0.0,0.333333,0.952130
8587,10248,Ростовская область,Таганрог,38.921404,47.270064,unknown,15,1046,87,4153763,...,0.825194,0.003169,0.000000,0.000000,0.0,0.0,0.0,0.0,0.333333,1.144000
8588,22502,Ростовская область,посёлок Матвеев Курган,38.859816,47.565717,residential,12,1905,87,4153763,...,0.886482,0.003046,0.262295,0.186567,0.0,0.0,0.0,0.5,0.333333,1.144000
8589,24497,Москва,Москва,37.720271,55.597304,residential,12,682,56,12635466,...,0.163820,0.026330,0.071942,0.070175,0.0,0.0,0.0,0.0,0.047619,1.022860
8590,23685,Ростовская область,село Авило-Успенка,38.674475,47.692474,unknown,15,2871,87,4153763,...,0.960914,0.008168,0.016393,0.014925,0.0,0.0,0.0,0.0,0.333333,1.144000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26012,18085,Московская область,Воскресенск,38.693206,55.310078,industrial,7,215,57,7768878,...,0.032874,0.062036,0.030928,0.023392,0.0,0.0,0.0,0.0,0.000000,0.952130
26013,23428,Ереван,Ереван,44.525006,40.131853,residential,12,325,26,1092000,...,0.373645,0.061169,0.100629,0.038902,0.0,0.0,0.0,0.0,0.000000,1.363095
26014,11862,Лорийская область,Ванадзор,44.483527,40.811495,residential,12,170,50,212600,...,0.000000,0.000000,0.673469,0.531532,0.0,0.0,0.0,1.0,1.000000,0.407616
26015,10382,Ереван,Ереван,44.525365,40.207058,residential,12,325,26,1092000,...,0.360082,0.002178,0.144654,0.164760,0.0,0.0,0.0,0.0,0.000000,1.363095


In [144]:
#data.drop('Nearest_Neighbor', axis = 1, inplace=True)
data.drop('type', axis = 1, inplace=True)

<ipython-input-144-e1dd105a2f16>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('type', axis = 1, inplace=True)


In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from catboost import CatBoostRegressor

In [146]:
data.columns

Index(['id', 'sales_per_month', 'region', 'locality', 'x', 'y', 'fclass',
       'fclass_encoded', 'locality_encoded', 'region_encoded', 'Population',
       'unemployment_lvl', 'salary', 'migration_rate', 'country',
       'pop_iso3_per_region', 'pop_iso5_per_region', 'pop_iso10_per_region',
       'pop_iso15_per_region', 'pop_iso30_per_region', 'country_encoded',
       'people_iso_15_on_people_iso_30_normalized',
       'people_iso_10_on_people_iso_30_normalized',
       'people_iso_3_on_people_iso_30_normalized',
       'people_iso_10_on_people_iso_15_normalized',
       'people_iso_5_on_people_iso_15_normalized',
       'people_iso_3_on_people_iso_15_normalized',
       'people_iso_5_on_people_iso_10_normalized',
       'people_iso_3_on_people_iso_10_normalized',
       'people_iso_3_on_people_iso_5_normalized',
       'people_iso_30_min_people_iso_15_normalized',
       'people_iso_15_min_people_iso_10_normalized',
       'people_iso_10_min_people_iso_5_normalized',
       'peopl

In [147]:
#X = data.iloc[:,7:]
columns = ['id', 'fclass_encoded', 'Population',
       'unemployment_lvl', 'salary', 'migration_rate',
       'pop_iso3_per_region', 'pop_iso5_per_region', 'pop_iso10_per_region',
       'pop_iso15_per_region', 'pop_iso30_per_region', 'country_encoded',
       'people_iso_15_on_people_iso_30_normalized',
       'people_iso_10_on_people_iso_30_normalized',
       'people_iso_3_on_people_iso_30_normalized',
       'people_iso_10_on_people_iso_15_normalized',
       'people_iso_5_on_people_iso_15_normalized',
       'people_iso_3_on_people_iso_15_normalized',
       'people_iso_5_on_people_iso_10_normalized',
       'people_iso_3_on_people_iso_10_normalized',
       'people_iso_3_on_people_iso_5_normalized',
       'people_iso_30_min_people_iso_15_normalized',
       'people_iso_15_min_people_iso_10_normalized',
       'people_iso_10_min_people_iso_5_normalized',
       'people_iso_10_min_people_iso_3_normalized',
       'people_iso_5_min_people_iso_3_normalized',
       'people_iso_30_deriv_people_iso_15',
       'people_iso_15_deriv_people_iso_10', 'people_iso_10_deriv_people_iso_5',
       'people_iso_10_deriv_people_iso_3', 'people_iso_5_deriv_people_iso_3',
       'people_iso_30_normalized', 'people_iso_15_normalized',
       'people_iso_10_normalized', 'people_iso_5_normalized',
       'people_iso_3_normalized', 'count_pvz_iso_3_normalized',
       'count_pvz_iso_5_normalized', 'count_pvz_iso_10_normalized',
       'count_pvz_iso_15_normalized', 'count_pvz_iso_30_normalized',
       'pop_per_pvz_iso3_normalized', 'pop_per_pvz_iso5_normalized',
       'pop_per_pvz_iso10_normalized', 'pop_per_pvz_iso15_normalized',
       'pop_per_pvz_iso30_normalized', 'Nearest_Neighbor_normalized',
       'Distance_normalized', 'commercial_density_300_normalized',
       'commercial_density_500_normalized',
       'count_railway_station_iso_3_normalized',
       'count_railway_station_iso_5_normalized',
       'count_railway_station_iso_10_normalized',
       'count_railway_station_iso_15_normalized',
       'count_railway_station_iso_30_normalized', 'std_ratio']#, 'locality_encoded', 'region_encoded']
X = data[columns]
y = data['sales_per_month']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




In [148]:
X_train.iloc[:,1:]

,fclass_encoded,Population,unemployment_lvl,salary,migration_rate,pop_iso3_per_region,pop_iso5_per_region,pop_iso10_per_region,pop_iso15_per_region,pop_iso30_per_region,...,Nearest_Neighbor_normalized,Distance_normalized,commercial_density_300_normalized,commercial_density_500_normalized,count_railway_station_iso_3_normalized,count_railway_station_iso_5_normalized,count_railway_station_iso_10_normalized,count_railway_station_iso_15_normalized,count_railway_station_iso_30_normalized,std_ratio
5121,15,1462021,3.1,41014.70,38.651976,0.000000,0.325577,0.909700,2.751670,10.492325,...,0.426920,0.030271,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.798329
58,15,7768878,2.7,70704.60,89.500000,0.000000,0.000386,0.004634,0.028576,0.121639,...,0.283471,0.156434,0.010309,0.005848,0.0,0.0,0.0,0.0,0.000000,0.952130
2301,12,1168771,3.8,40804.10,-19.900000,0.331973,1.919110,9.026576,21.792122,92.881326,...,0.693819,0.007464,0.090909,0.155556,0.0,0.0,0.0,0.0,0.000000,0.960264
6808,15,1198429,4.7,41527.00,-42.600000,0.028370,0.549052,3.584693,9.087731,15.905823,...,0.735287,0.014554,0.133333,0.306122,0.0,0.0,0.0,0.0,1.000000,0.886755
5113,12,7768878,2.7,70704.60,89.500000,0.072082,0.612830,1.300445,2.234042,10.649543,...,0.507006,0.010177,0.051546,0.052632,0.0,0.0,0.0,0.0,0.333333,0.952130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,12,2287678,3.6,46276.70,2.700000,0.001748,0.041090,0.274514,0.528921,2.094700,...,0.982267,0.048115,0.053763,0.024752,0.0,0.0,0.0,0.0,0.500000,0.911759
5191,15,4001678,4.4,49459.70,-3.400000,0.077717,0.378841,1.389417,1.697288,2.961258,...,0.186739,0.020147,0.016667,0.006452,0.0,0.0,0.0,0.0,0.000000,0.976545
5390,15,2268179,5.8,39270.40,-36.300000,0.000000,0.626053,3.214914,4.422931,5.617722,...,0.840804,0.005110,0.512195,0.435294,0.0,0.0,0.0,0.0,0.000000,1.057415
860,12,251200,21.9,24546.45,11.133228,2.452229,12.308917,37.078025,117.607484,211.110669,...,0.627343,0.021796,0.230769,0.555556,0.0,0.0,0.0,0.0,1.000000,0.927010


In [149]:
lr = LinearRegression()
lr.fit(X_train.iloc[:,1:], y_train)
lr_pred = lr.predict(X_test.iloc[:,1:])
r2_score(y_test, lr_pred)

0.12166993198353304

In [150]:
from sklearn.linear_model import Lasso, Ridge

ls = Lasso(tol = 0.00005, alpha = 0.2)
ls = ls.fit(X_train.iloc[:,1:], y_train)
ls_pred = ls.predict(X_test.iloc[:,1:])
r2_score(y_test, ls_pred)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.005e+09, tolerance: 1.176e+05
  model = cd_fast.enet_coordinate_descent(


0.12340350603476824

In [128]:
svr = SVR(kernel='rbf', C=1, epsilon=0.1)  # RBF kernel is commonly used for SVR

# Fit the model
svr = svr.fit(X_train, y_train)
svr_pred = svr.predict(X_test)
r2_score(y_test, svr_pred)

-0.014286308858769914

In [129]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=40, random_state=42)  # Set maximum depth of the tree
tree.fit(X_train, y_train)

# Make predictions
y_pred = tree.predict(X_test)
r2_score(y_pred, y_test)

-0.6899935515162716

In [130]:
model = CatBoostRegressor()

model.fit(X_train, y_train)
y_pred = model.predict(X_test)

r2_score(y_pred, y_test)

Learning rate set to 0.055514
0:	learn: 581.9616718	total: 4.09ms	remaining: 4.09s
1:	learn: 578.9367165	total: 12ms	remaining: 6.01s
2:	learn: 576.6829506	total: 16.6ms	remaining: 5.5s
3:	learn: 574.1703939	total: 22.3ms	remaining: 5.54s
4:	learn: 571.1652228	total: 25.2ms	remaining: 5.02s
5:	learn: 568.8331015	total: 27.7ms	remaining: 4.59s
6:	learn: 566.9317295	total: 30.6ms	remaining: 4.34s
7:	learn: 564.9735515	total: 34ms	remaining: 4.21s
8:	learn: 562.9378125	total: 36.6ms	remaining: 4.03s
9:	learn: 561.1380222	total: 39.4ms	remaining: 3.9s
10:	learn: 559.4536611	total: 42ms	remaining: 3.78s
11:	learn: 557.7214361	total: 45.2ms	remaining: 3.73s
12:	learn: 556.0406952	total: 48ms	remaining: 3.65s
13:	learn: 554.2622824	total: 50.5ms	remaining: 3.56s
14:	learn: 552.6763302	total: 53.3ms	remaining: 3.5s
15:	learn: 551.4003511	total: 56.2ms	remaining: 3.46s
16:	learn: 550.0596955	total: 59.1ms	remaining: 3.42s
17:	learn: 548.8977494	total: 61.9ms	remaining: 3.38s
18:	learn: 547.7443

182:	learn: 482.0269849	total: 580ms	remaining: 2.59s
183:	learn: 481.7508206	total: 586ms	remaining: 2.6s
184:	learn: 481.3234344	total: 589ms	remaining: 2.59s
185:	learn: 481.1357850	total: 593ms	remaining: 2.6s
186:	learn: 480.9215685	total: 596ms	remaining: 2.59s
187:	learn: 480.6589716	total: 599ms	remaining: 2.59s
188:	learn: 480.2764850	total: 602ms	remaining: 2.58s
189:	learn: 480.0120439	total: 605ms	remaining: 2.58s
190:	learn: 479.6523992	total: 608ms	remaining: 2.58s
191:	learn: 479.2878910	total: 611ms	remaining: 2.57s
192:	learn: 479.0542744	total: 614ms	remaining: 2.56s
193:	learn: 478.8253509	total: 616ms	remaining: 2.56s
194:	learn: 478.5785700	total: 619ms	remaining: 2.55s
195:	learn: 478.2047048	total: 621ms	remaining: 2.55s
196:	learn: 477.9871175	total: 623ms	remaining: 2.54s
197:	learn: 477.5622591	total: 626ms	remaining: 2.54s
198:	learn: 477.3672313	total: 629ms	remaining: 2.53s
199:	learn: 477.1160542	total: 632ms	remaining: 2.53s
200:	learn: 476.8364267	total:

396:	learn: 430.6012939	total: 1.16s	remaining: 1.77s
397:	learn: 430.4044046	total: 1.17s	remaining: 1.77s
398:	learn: 430.2438182	total: 1.17s	remaining: 1.77s
399:	learn: 429.9781684	total: 1.18s	remaining: 1.77s
400:	learn: 429.6984284	total: 1.18s	remaining: 1.76s
401:	learn: 429.5366754	total: 1.18s	remaining: 1.76s
402:	learn: 429.2412844	total: 1.19s	remaining: 1.75s
403:	learn: 428.9594952	total: 1.19s	remaining: 1.75s
404:	learn: 428.7321546	total: 1.19s	remaining: 1.75s
405:	learn: 428.6214835	total: 1.19s	remaining: 1.75s
406:	learn: 428.3003595	total: 1.2s	remaining: 1.74s
407:	learn: 427.9881503	total: 1.2s	remaining: 1.74s
408:	learn: 427.7222387	total: 1.2s	remaining: 1.74s
409:	learn: 427.4434503	total: 1.2s	remaining: 1.73s
410:	learn: 427.2875238	total: 1.21s	remaining: 1.73s
411:	learn: 427.0096159	total: 1.21s	remaining: 1.73s
412:	learn: 426.7476144	total: 1.21s	remaining: 1.72s
413:	learn: 426.5553508	total: 1.21s	remaining: 1.72s
414:	learn: 426.2803515	total: 1

594:	learn: 391.8646872	total: 1.75s	remaining: 1.19s
595:	learn: 391.7286437	total: 1.75s	remaining: 1.19s
596:	learn: 391.6133051	total: 1.75s	remaining: 1.18s
597:	learn: 391.3862764	total: 1.76s	remaining: 1.18s
598:	learn: 391.1518651	total: 1.76s	remaining: 1.18s
599:	learn: 390.9944726	total: 1.76s	remaining: 1.18s
600:	learn: 390.7874893	total: 1.76s	remaining: 1.17s
601:	learn: 390.6029756	total: 1.77s	remaining: 1.17s
602:	learn: 390.3027258	total: 1.77s	remaining: 1.17s
603:	learn: 390.0676407	total: 1.77s	remaining: 1.16s
604:	learn: 389.8872423	total: 1.78s	remaining: 1.16s
605:	learn: 389.6907784	total: 1.78s	remaining: 1.16s
606:	learn: 389.5812174	total: 1.78s	remaining: 1.15s
607:	learn: 389.3730599	total: 1.79s	remaining: 1.15s
608:	learn: 389.1228700	total: 1.79s	remaining: 1.15s
609:	learn: 389.0207565	total: 1.79s	remaining: 1.15s
610:	learn: 388.8369110	total: 1.79s	remaining: 1.14s
611:	learn: 388.5848551	total: 1.8s	remaining: 1.14s
612:	learn: 388.3076205	total

782:	learn: 359.0889207	total: 2.33s	remaining: 645ms
783:	learn: 358.8956832	total: 2.33s	remaining: 642ms
784:	learn: 358.7665504	total: 2.33s	remaining: 639ms
785:	learn: 358.6048721	total: 2.34s	remaining: 637ms
786:	learn: 358.4243782	total: 2.34s	remaining: 634ms
787:	learn: 358.2842696	total: 2.34s	remaining: 631ms
788:	learn: 358.1366657	total: 2.35s	remaining: 628ms
789:	learn: 357.9103279	total: 2.35s	remaining: 625ms
790:	learn: 357.6815560	total: 2.35s	remaining: 622ms
791:	learn: 357.6157917	total: 2.35s	remaining: 618ms
792:	learn: 357.4396393	total: 2.36s	remaining: 615ms
793:	learn: 357.2713117	total: 2.36s	remaining: 612ms
794:	learn: 357.1185557	total: 2.36s	remaining: 609ms
795:	learn: 357.0014116	total: 2.37s	remaining: 606ms
796:	learn: 356.8136994	total: 2.37s	remaining: 603ms
797:	learn: 356.6496374	total: 2.37s	remaining: 600ms
798:	learn: 356.5790480	total: 2.37s	remaining: 597ms
799:	learn: 356.4457609	total: 2.38s	remaining: 594ms
800:	learn: 356.2907971	tota

991:	learn: 328.1227479	total: 2.91s	remaining: 23.5ms
992:	learn: 328.0013875	total: 2.92s	remaining: 20.6ms
993:	learn: 327.9312443	total: 2.92s	remaining: 17.6ms
994:	learn: 327.8104784	total: 2.92s	remaining: 14.7ms
995:	learn: 327.6829985	total: 2.93s	remaining: 11.8ms
996:	learn: 327.4929602	total: 2.93s	remaining: 8.81ms
997:	learn: 327.3637265	total: 2.93s	remaining: 5.88ms
998:	learn: 327.1892027	total: 2.93s	remaining: 2.94ms
999:	learn: 327.0407138	total: 2.94s	remaining: 0us


-2.5639902633638116

In [151]:
#final_pred = ls.predict(test.iloc[:,-18:])
final_pred = ls.predict(test[columns].iloc[:,1:])
#final_pred = lr.predict(test[columns])

In [152]:
test['sales_per_month'] = final_pred

<ipython-input-152-c25a0af523fc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sales_per_month'] = final_pred


In [153]:
test[['id', 'sales_per_month']].to_csv('pred.csv', index=False)